In [24]:
import os
os.chdir("Wine-Quality-Prediction")
%pwd

'c:\\Users\\sneha\\OneDrive\\Desktop\\wine quality prediction\\Wine-Quality-Prediction'

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [27]:
from src.wine.constants import *
from src.wine.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [30]:
import os
import urllib.request as request
from src.wine.utils import logger
import zipfile

In [32]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [33]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-15 00:27:50,914:INFO:common:Reading the yaml file from the path: config\config.yaml]
[2025-01-15 00:27:50,940:INFO:common:Reading the yaml file from the path: params.yaml]
[2025-01-15 00:27:50,942:INFO:common:Reading the yaml file from the path: schema.yaml]
[2025-01-15 00:27:50,947:INFO:common:Creating directory: artifacts]
[2025-01-15 00:27:50,950:INFO:common:Creating directory: artifacts/data_ingestion]
[2025-01-15 00:27:52,746:INFO:3544816806:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7A6F:A4B3C:6FE97:9A13A:6786B3AF
Accept-Ranges: bytes
Date: Tue, 14 Jan 20